In [1]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
Path='/home/mgander/Atlantic/data/copied_data/Atlantic'

In [3]:
#file='fNCI60'
#file='fNCI60_GluC'
#file='fCRC65'

#file='pNCI60'
#file='pNCI60_GluC'
#file='pCRC65'

In [4]:
file='pNCI60'
df0=pd.read_pickle(f'{Path}/{file}.pkl')

In [5]:
file='pCRC65'
df1=pd.read_pickle(f'{Path}/{file}.pkl')

In [7]:
len(sorted(set(df0['Experiment'])))

60

In [8]:
len(sorted(set(df1['Experiment'])))

65

# Reshape dataframe into matrix

In [9]:
df=pd.concat([df0, df1], ignore_index=True)
df=df[~np.isnan(df['Intensity'])].copy()

In [13]:
mods=list(df['Modified sequence'])
prots=list(df['Proteins'])
D_mod_to_prot={mods[i]:prots[i] for i in range(len(mods))}

np.save(f"{Path}/D_mod_to_prot.npy", D_mod_to_prot)

In [9]:
# Some intensities are nan, which is a little weird

In [10]:
mods=sorted(set(df['Modified sequence']))
exps=sorted(set(df['Experiment']))

In [11]:
D_mod_to_ind={mods[i]:i for i in range(len(mods))}

In [12]:
M=np.full((len(mods), len(exps)), np.NaN)

In [13]:
for i,exp in enumerate(exps):
    print(i)
    dfc=df[df['Experiment']==exp].copy()
    dfcs=dfc[['Modified sequence', 'Intensity']].copy()
    ints=dfcs.groupby('Modified sequence').mean()
    indices=[D_mod_to_ind[a] for a in ints.index]
    M[indices, i:i+1]=np.array(ints.values)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [15]:
Df=pd.DataFrame(data=M, columns=exps, index=mods)

In [18]:
#Df.to_pickle(f'{Path}/P_M_not_normalized.pkl')

In [15]:
for i in range(len(exps)):
    Df[exps[i]]=Df[exps[i]]/np.nanmedian(Df[exps[i]])

# Combine cell lines that were measured several times

In [16]:
def purge_and_capitalize(ccl):
    ccl_str=ccl.replace('-', '')
    ccl_str=ccl_str.replace(' ', '')
    return(ccl_str.upper())

def map_celllines_to_cellosaurus(ccls):
    # Let's remove '-' and spaces and capitalize everything
    D_ccl={}

    for ccl in ccls:
        D_ccl[ccl]=purge_and_capitalize(ccl)
    ccl_red=sorted(set([D_ccl[a] for a in ccls]))
    len(ccl_red)
    
    Path='/home/mgander/Atlantic/data/'
    dfc=pd.read_pickle(f'{Path}/Viability/CCL_dict/Cellosaurus_CCLs.pkl')
    dfc=dfc[dfc['Species']=='NCBI_TaxID=9606; ! Homo sapiens (Human)']

    # Use the synonyms to map to the Cellosaurus ID
    cellosaurus_ids=list(dfc['Cellosaurus_ID'])
    syns=list(dfc['Synonyms'])

    syn_l=[]
    for s in syns:
        if '; ' in s:
            syn_l.append(s.split('; '))
        else:
            syn_l.append(s)
    D_cello={cellosaurus_ids[i]:cellosaurus_ids[i] for i in range(len(cellosaurus_ids))}

    for i in range(len(syn_l)):
        for s in syn_l[i]:
            D_cello[s]=cellosaurus_ids[i]
            
    ccl_red=sorted(set(ccl_red))
    for ccl in ccl_red:
        if ccl in cellosaurus_ids:
            D_cello[ccl]=ccl
            
    D_cello['C106']='C106[HUMANRECTALADENOCARCINOMA]'
    D_cello['CC20']='CC20[HUMANCOLONADENOCARCINOMA]'
    D_cello['NCIADRES']='NCIADRRES'
    D_cello['Hela']='HELA'
    
    keys=list(D_cello.keys())
    # Still missing
    for ccl_red in ccl_red:
        if not ccl_red in keys:
            print(f'{ccl_red}')
    #return([D_ccl[a] for a in ccls])
    
    return([D_cello[D_ccl[a]] for a in ccls])

In [17]:
dfe=pd.DataFrame({'Experiment':exps})
dfe['CCL']=[a.split('_')[0] for a in exps]
dfe['Cello']=map_celllines_to_cellosaurus(list(dfe['CCL']))

In [18]:
dvc=dfe['Cello'].value_counts()
plurals=list(dvc[dvc>1].index)
singulars=list(dvc[dvc==1].index)

In [19]:
Dfs=Df[list(dfe[dfe['Cello'].isin(singulars)]['Experiment'])].copy()
Dfs.columns=[a.split('_')[0] for a in singulars]

for p in plurals:
    cols=list(dfe[dfe['Cello']==p]['Experiment'])
    dfs=Df[cols]
    Dfs[p]=np.nanmean(dfs.values, 1)

/tmp/ipykernel_3420/3241054075.py:7: RuntimeWarning: Mean of empty slice
  Dfs[p]=np.nanmean(dfs.values, 1)
/tmp/ipykernel_3420/3241054075.py:7: RuntimeWarning: Mean of empty slice
  Dfs[p]=np.nanmean(dfs.values, 1)
/tmp/ipykernel_3420/3241054075.py:7: RuntimeWarning: Mean of empty slice
  Dfs[p]=np.nanmean(dfs.values, 1)


In [42]:
# Check with Cellosaurus
dfc=pd.read_pickle(f'/home/mgander/Atlantic/data/Viability/CCL_dict/Cellosaurus_CCLs.pkl')
for a in Dfs.columns:
     if not a in list(dfc['Cellosaurus_ID']):
            print(a)

In [43]:
Dfs.index=['PP'+a for a in Dfs.index]

In [44]:
Dfs.columns=['Atlantic_?_?_'+a+'_?_0um_0min' for a in Dfs.columns]

In [45]:
Dfs.to_pickle(f'{Path}/P_M.pkl')